# Notebook for the development of module "scheduling_problem" of AI for carbon reduction project

# General model
We list below the common sets, parameters, variables and constraints that will be used in all the models.

## Sets
$$
T = \{0, ..., T_H-1\} = \text{Discrete time steps in the scheduling horizon (e.g., minutes).}
$$
$$
I = \{1, ...., N_M\} = \text{Set of machines.}
$$
$$
J_i = \{1, ..., N_i\} = \text{Set of jobs to be processed by machine $i$.}
$$

Other sets that will be used in the constraints:

$SHARED = \{M'$ : if $k,l \in M'$ then $k$ and $l$ have shared resources$\}$

$DEPEN = \{(k,l)$ : $l$ must start its job only after $k$ has finished its$\}$


## Parameters
For each machine:
$$
 e_{i} = \text{energy used by the machine $i$ per unit of time}
 $$
 $$
 f_{i} = \text{fixed cost for starting the machine $i$ at any time}
 $$
 $$
d_i = \text{duration of any of the job of machine $i$}
$$
$$
N_i = \text{number of jobs to complete for machine $i$}
$$
$$
c_i = \text{time of cooldown for machine $i$}
$$
$$
T_{j,i} = \text{threshold for job $j$ of machine $i$}
$$
For each time step:
$$
p_t = \text{energy produced at time $t$ by one solar panel}
$$
$$
m_t = \text{maximum energy that we can use at time $t$}
$$
For each solar panel:
$$
c_p = \text{cost of a solar panel}
$$
For each battery:
$$
c_b = \text{cost of a battery}
$$
$$
B = \text{capacity of a battery}
$$

## Variables
$$
x_{itj} = \text{if machine $i$ is working at time $t$ for job $j$}
$$
$$
y_{itj} = \text{if machine $i$ starts working at time $t$ for job $j$}
$$
$$
z_{t} = \text{value of objective function at time t, real number}
$$

## Constraints

1. Every machine $i \in I$ must do the required number $N_i$ of jobs with a certain duration $d_i$. For every $i \in I$ and $j \in J_i$:
$$
\sum_{t \in T} x_{itj} \ge  d_i
$$
2. Knapsack contraint that says that not all machines can run at the same time due to max power load for slot (parameter $m_t$, which could depend on time). For every $t \in T$:
$$
   \sum_{i \in I} \sum_{j \in J_i} e_{i}x_{itj} \le m_t
$$
3. Some machines may be allowed to run only at specific times for noise or maintenance windows / downtime or worker availability. For some $t \in T$ and $i \in I$:
   $$
   x_{itj} = 0 \, \, \forall j \in J_i
   $$
4. Shared resources: some machines can't run in the same time slot $t \in T$ because of shared resources. Given a set $SHARED = \{M'$ : if $k,l \in M'$ then $k$ and $l$ have shared resources$\}$ and $\forall t \in T$:
   $$
   \sum_{i \in M'} \sum_{j \in J_i} x_{itj} \le 1  \,\, \forall M' \in SHARED
   $$
5. A machine works only if it has just started or it was already working. For every $i \in I$, for every $j \in J_i$ and for every $t \in T$:
$$            
y_{itj} \le x_{itj} \le y_{itj}+x_{i(t-1)j}
$$
6. Machine dependencies: one machine starts only after another has finished. Given a set $DEPEN = \{(k,l)$ : $l$ must start its job only after $k$ has finished its$\}$ and for every time $t \in T$.
$$
y_{ltj} \cdot d_k \le  \sum_{t' < t, \, t' \in T} (x_{kt'j}) \text{ for every dependency }(k,l) \in DEPEN \text{ and for every $j \in J_l$}
$$
7. Cooldown Periods: some machines require a cooldown phase between uses (if this is not needed, set $c_i = 0$). For every $i \in I$ and for every $t \in T$:
$$
   c_i \cdot \sum_{j \in J_i} y_{itj} \le \sum_{t-c_i \le t' < t, \, t' \in T}\sum_{j \in J_i} (1-x_{it'j})
$$
8. Job deadlines: all jobs must finish before a specific time. For every $i \in I$ and for every $j \in J_i$, calling $T_{j,i}$ the threshold for job $j$ on machine $i$:
$$
     \sum_{t' \le T_{j,i}, \, t' \in T} x_{it'j} \ge d_i
$$
9. Only one job at a time. For every $i \in I$ and for every $t \in T$:
$$
    \sum_{j \in J_i} x_{itj} \le 1
$$
$$
    \sum_{j \in J_i} y_{itj} \le 1
$$


## PHASE 1
Determine the minimum number of solar panels ($M$) and battery units ($N$) for 100% self-sufficiency.

### Additional variables
$$
N = \text{how many batteries to buy}
$$
$$
M = \text{how many panels to buy}
$$
$$
s_t = \text{how much energy we accumulated till time $t$ in the batteries}
$$
### Objective function
$$
\min Nc_b+Mc_p
$$
### Additional constraints
1. No imported energy. For every $t \in T$:
$$
\sum_{i \in I}\sum_{j \in J_i} (e_{i}x_{itj} +f_i y_{itj} - Mp_t - s_t) \le  0
$$
2. Constraints on the level of energy accumulated in the batteries. For every $t \in T$:
$$
         s_t \le  \sum_{t'<t, \, t' \in T}\sum_{i \in I}\sum_{j \in J} (Mp_{t'} - e_{i}x_{it'j} - f_i y_{it'j})
$$
$$
         0 \le s_t \le  NB
$$

### Final model
The following is the final model that is solved in phase 1.

$$
\left\{
  \begin{array}{rcr}
        \min Nc_b+Mc_p \\
        \sum_{i \in I}\sum_{j \in J_i} (e_{i}x_{itj} +f_i y_{itj}) - Mp_t - s_t \le & 0 & \forall t \in T \\
        s_t  \le & \sum_{t'<t, \, t' \in T}\sum_{i \in I}\sum_{j \in J} (Mp_{t'} - e_{i}x_{it'j} - f_i y_{it'j}) & \forall t \in T \\
        0   \le s_t \le & NB & \forall t \in T \\
        \sum_{t \in T} x_{itj} \ge & d_i & \forall i \in I, \, \forall j \in J \\
        \sum_{i \in I} \sum_{j \in J_i} e_{i}x_{itj}  \le & m_t & \forall t \in T \\
        x_{itj} = & 0 & \text{for some $t \in T$ and for some $i \in I$, } \forall j \in J_i \\
        \sum_{i \in M'} \sum_{j \in J_i} x_{itj} \le & 1  & \forall t \in T, \, \forall M' \in SHARED\\
        y_{itj} \le x_{itj} \le & y_{itj}+x_{i(t-1)j} & \forall t \in T-\{0\}, \, \forall i \in I, \, \forall j \in J_i \\
        y_{ltj} \cdot d_k  \le & \sum_{t' < t, \, t' \in T} (x_{kt'j}) & \text{ for every dependency }(k,l) \in M \text{ and $\forall j \in J_l$, $\forall t \in T$} \\
        c_i \cdot \sum_{j \in J_i} y_{itj} \le & \sum_{t-c_i \le t' < t, \, t' \in T}\sum_{j \in J_i} (1-x_{it'j}) & \forall t \in T, \, \forall i \in I \\
        \sum_{t' \le T_{j,i}, \, t' \in T} x_{it'j} \ge & d_i & \forall i \in I, \, \forall j \in J_i\\
        \sum_{j \in J_i} x_{itj} \le & 1 & \forall t \in T, \, \forall i \in I\\
        \sum_{j \in J_i} y_{itj} \le & 1 & \forall t \in T, \, \forall i \in I
  \end{array}
\right.
$$

## PHASE 2
Taking the number of panels $M^*$ and the number of batteries $N^*$ that result from phase 1, compute the number of years that our company could work continuing to pay external energy instead of buying solar panels.

Let $c_{annual}$ be the annual cost of the imported energy.

$$
horizon = H = \frac{c_bN^*+c_pM^*}{c_{annual}}
$$

## PHASE 3
Within the budget implied by Phase 2’s horizon, find optimal $N$ and $M$ to minimize total energy cost (capital + operational).

### Additional parameters
$$
T_H = H_{time \, units} = \text{temporal horizon from phase 2 in time units} 
$$
$$
BIG_M = \text{Big number}
$$
### Additional variables
$$
z_t = \text{imported energy at time t}
$$
$$
V_t = \text{represents the difference between the energy that has been produced by our solar banels and the energy that has been used}
$$
$$
b_t = \text{binary variable that is 1 if $V_t < 0$, 0 otherwise}
$$

### Objective function
$$
\min Nc_b + Mc_p + \sum_{0\le t\le H, \, t \in T} z_t
$$
### Additional constraints
1. Modified energy balance constraint. For every $t \in T$:
$$
\sum_{i \in I} \sum_{j \in J_i} (e_{i}x_{itj} +f_i y_{itj}) - Mp_t - s_t \le z_t
$$
2. Definition of $V_t$ for every $t \in T$:
$$
V_t = \sum_{t'<t, \, t' \in T}\sum_{i \in I}\sum_{j \in J_i} (Mp_{t'} - e_{i}x_{it'j} - f_i y_{it'j})
$$
3. Bounds on $V_t$ depending on $b_t$:
$$
V_t \ge -BIG_M (1 - b_t)
$$
$$
V_t \le BIG_M b_t
$$
4. Bounds on $s_t$ that depend on $V_t$ and $b_t$:
$$
s_t \le V_t + BIG_M (1 - b_t)
$$
$$
s_t \le BIG_M b_t
$$
$$
0 \le s_t \le NB
$$
4. Bound on $z_t$:
$$
0 \le z_t
$$

### Final model
$$
\left\{
  \begin{array}{rcr}
        \min Nc_b + Mc_p + \sum_{t \in T} z_t \\
        \sum_{i \in I} \sum_{j \in J_i} (e_{i}x_{itj} +f_i y_{itj}) - Mp_t - s_t \le & z_t & \forall t \in T \\
        V_t = & \sum_{t'<t, \, t' \in T}\sum_{i \in I}\sum_{j \in J_i} (Mp_{t'} - e_{i}x_{it'j} - f_i y_{it'j}) & \forall t \in T \\
        V_t \ge & -BIG_M (1 - b_t) & \forall t \in T \\
        V_t \le & BIG_M b_t & \forall t \in T \\
        s_t \le & V_t + BIG_M (1 - b_t) & \forall t \in T \\
        s_t \le & BIG_M b_t & \forall t \in T \\
        0 \le s_t \le & NB & \forall t \in T \\
        z_t \ge & 0 & \forall t \in T \\
        \sum_{t \in T} x_{itj} \ge & d_i & \forall i \in I, \, \forall j \in J \\
        \sum_{i \in I} \sum_{j \in J_i} e_{i}x_{itj}  \le & m_t & \forall t \in T \\
        x_{itj} = & 0 & \text{for some $t \in T$ and for some $i \in I$, } \forall j \in J_i \\
        \sum_{i \in M'} \sum_{j \in J_i} x_{itj} \le & 1  & \forall t \in T, \, \forall M' \in SHARED\\
        y_{itj} \le x_{itj} \le & y_{itj}+x_{i(t-1)j} & \forall t \in T-\{0\}, \, \forall i \in I, \, \forall j \in J_i \\
        y_{ltj} \cdot d_k  \le & \sum_{t' < t, \, t' \in T} (x_{kt'j}) & \text{ for every dependency }(k,l) \in M \text{ and $\forall j \in J_l$, $\forall t \in T$} \\
        c_i \cdot \sum_{j \in J_i} y_{itj} \le & \sum_{t-c_i \le t' < t, \, t' \in T}\sum_{j \in J_i} (1-x_{it'j}) & \forall t \in T, \, \forall i \in I \\
        \sum_{t' \le T_{j,i}, \, t' \in T} x_{it'j} \ge & d_i & \forall i \in I, \, \forall j \in J_i\\
        \sum_{j \in J_i} x_{itj} \le & 1 & \forall t \in T, \, \forall i \in I\\
        \sum_{j \in J_i} y_{itj} \le & 1 & \forall t \in T, \, \forall i \in I
  \end{array}
\right.
$$

## PHASE 4
Now that we fixed N and M, we need to obtain an optimal scheduling that reduces the usage of external energy.

This phase can be divided in 2.

### 4.1
If I need to not use at all any form of external energy. It's the same model of Phase 1 but without the objective function.

$$
\left\{
  \begin{array}{rcr}
        \sum_{i \in I}\sum_{j \in J_i} (e_{i}x_{itj} +f_i y_{itj}) - Mp_t - s_t \le & 0 & \forall t \in T \\
        s_t  \le & \sum_{t'<t, \, t' \in T}\sum_{i \in I}\sum_{j \in J} (Mp_{t'} - e_{i}x_{it'j} - f_i y_{it'j}) & \forall t \in T \\
        0   \le s_t \le & NB & \forall t \in T \\
        \sum_{t \in T} x_{itj} \ge & d_i & \forall i \in I, \, \forall j \in J \\
        \sum_{i \in I} \sum_{j \in J_i} e_{i}x_{itj}  \le & m_t & \forall t \in T \\
        x_{itj} = & 0 & \text{for some $t \in T$ and for some $i \in I$, } \forall j \in J_i \\
        \sum_{i \in M'} \sum_{j \in J_i} x_{itj} \le & 1  & \forall t \in T, \, \forall M' \in SHARED\\
        y_{itj} \le x_{itj} \le & y_{itj}+x_{i(t-1)j} & \forall t \in T-\{0\}, \, \forall i \in I, \, \forall j \in J_i \\
        y_{ltj} \cdot d_k  \le & \sum_{t' < t, \, t' \in T} (x_{kt'j}) & \text{ for every dependency }(k,l) \in M \text{ and $\forall j \in J_l$, $\forall t \in T$} \\
        c_i \cdot \sum_{j \in J_i} y_{itj} \le & \sum_{t-c_i \le t' < t, \, t' \in T}\sum_{j \in J_i} (1-x_{it'j}) & \forall t \in T, \, \forall i \in I \\
        \sum_{t' \le T_{j,i}, \, t' \in T} x_{it'j} \ge & d_i & \forall i \in I, \, \forall j \in J_i\\
        \sum_{j \in J_i} x_{itj} \le & 1 & \forall t \in T, \, \forall i \in I\\
        \sum_{j \in J_i} y_{itj} \le & 1 & \forall t \in T, \, \forall i \in I
  \end{array}
\right.
$$

### 4.3
Minimize the external energy used. It's the same as model 3 but with a modified objective function that doesn't aim to minimize N and M.

$$
\left\{
  \begin{array}{rcr}
        \min \sum_{t \in T} z_t \\
        \sum_{i \in I} \sum_{j \in J_i} (e_{i}x_{itj} +f_i y_{itj}) - Mp_t - s_t \le & z_t & \forall t \in T \\
        V_t = & \sum_{t'<t, \, t' \in T}\sum_{i \in I}\sum_{j \in J_i} (Mp_{t'} - e_{i}x_{it'j} - f_i y_{it'j}) & \forall t \in T \\
        V_t \ge & -BIG_M (1 - b_t) & \forall t \in T \\
        V_t \le & BIG_M b_t & \forall t \in T \\
        s_t \le & V_t + BIG_M (1 - b_t) & \forall t \in T \\
        s_t \le & BIG_M b_t & \forall t \in T \\
        0 \le s_t \le & NB & \forall t \in T \\
        z_t \ge & 0 & \forall t \in T \\
        \sum_{t \in T} x_{itj} \ge & d_i & \forall i \in I, \, \forall j \in J \\
        \sum_{i \in I} \sum_{j \in J_i} e_{i}x_{itj}  \le & m_t & \forall t \in T \\
        x_{itj} = & 0 & \text{for some $t \in T$ and for some $i \in I$, } \forall j \in J_i \\
        \sum_{i \in M'} \sum_{j \in J_i} x_{itj} \le & 1  & \forall t \in T, \, \forall M' \in SHARED\\
        y_{itj} \le x_{itj} \le & y_{itj}+x_{i(t-1)j} & \forall t \in T-\{0\}, \, \forall i \in I, \, \forall j \in J_i \\
        y_{ltj} \cdot d_k  \le & \sum_{t' < t, \, t' \in T} (x_{kt'j}) & \text{ for every dependency }(k,l) \in M \text{ and $\forall j \in J_l$, $\forall t \in T$} \\
        c_i \cdot \sum_{j \in J_i} y_{itj} \le & \sum_{t-c_i \le t' < t, \, t' \in T}\sum_{j \in J_i} (1-x_{it'j}) & \forall t \in T, \, \forall i \in I \\
        \sum_{t' \le T_{j,i}, \, t' \in T} x_{it'j} \ge & d_i & \forall i \in I, \, \forall j \in J_i\\
        \sum_{j \in J_i} x_{itj} \le & 1 & \forall t \in T, \, \forall i \in I\\
        \sum_{j \in J_i} y_{itj} \le & 1 & \forall t \in T, \, \forall i \in I
  \end{array}
\right.
$$

# Algorithms to solve the problem

### Local search
We can begin by solving a Constraint Satisfaction Problem (CSP) to obtain a feasible solution, and then apply local search techniques to optimize it further.

The basic idea of local search is to start from a feasible solution and explore its neighboring solutions. If a neighbor has a better value (e.g., higher utility or lower cost), we move to that neighbor. This approach is known as hill-climbing. However, a major limitation is that it can get stuck in local optima (either minima or maxima).

To address this, several strategies can be used:
   * Stochastic hill-climbing: Rather than always choosing the best neighbor, randomly select one from among the better neighbors.
   * Multiple restarts: Perform several hill-climbing runs from different starting points and return the best solution found.
   * Simulated annealing: At each step, select a random move. If the move improves the solution, accept it; if not, accept it with a probability that decreases over time. This method is particularly popular in applications like airline scheduling.
   * Local beam search: Start with k different initial states and, at each iteration, consider all their neighbors, selecting the best k to continue. The process repeats until a goal is found.
   * Genetic algorithms: Maintain a population of candidate solutions. New candidates are generated by combining existing ones (e.g., via crossover), and selection favors high-quality solutions.

### CSP
The goal is to assign one value to each variable such that all constraints are satisfied. Instead of relying on problem-specific strategies, we apply general-purpose heuristics.

A state represents an assignment of values to some or all variables. We aim to reach states that are both consistent (do not violate any constraints) and complete (all variables assigned).

Note: Any Constraint Satisfaction Problem (CSP) can be transformed into an equivalent CSP involving only binary constraints.

Solving CSPs:
   * BACKTRACKING SEARCH: Begin with an empty assignment and use depth-first search to assign values that don't cause conflicts. If no valid values are available, backtrack.
   * IMPROVED BACKTRACKING:
     * Variable Selection: Choose the variable with the fewest legal values. If tied, prefer the one involved in the most constraints with unassigned variables.
     * Value Selection: For the selected variable, pick the value that eliminates the fewest options for other variables (i.e., the least constraining value).
     * Backjumping: Instead of backtracking to the most recent assignment, backtrack directly to the variable most likely to resolve the conflict.
     * No-Good Recording: Learn from failures by recording combinations of variable assignments that lead to dead ends, preventing repeated mistakes.
     * Forward Checking: After each assignment, eliminate inconsistent values from the domains of remaining variables. If any domain becomes empty, backtrack.
     * Constraint Propagation: Enforce local consistency to prune the search space:
       * Node Consistency: Each value in a variable’s domain satisfies all its unary constraints.
       * Arc Consistency (e.g., AC-3 algorithm): Every value in a variable’s domain is consistent with some value in the connected variable’s domain under binary constraints.
       * Path Consistency: Ensures consistency across chains involving more than two variables.
### Local search for CSP
This approach begins with a complete assignment that may violate some constraints, then makes small changes—typically modifying the value of one variable at a time—to move toward a valid solution. The search space consists of all complete assignments, and the goal is to find one that is consistent (violates no constraints).

This method is especially helpful in dynamic scenarios like scheduling, where an initial good solution exists, but small changes are needed due to new constraints or updates.

To guide the search, each state is evaluated by counting the number of violated constraints.

Techniques:
 * Min-Conflicts Heuristic: At each step, select a new value for a variable that results in the fewest constraint violations.
 * Improvements and Variants:
   * Random Restarts: If no solution is found after a set number of steps, restart the process from a new random assignment.
   * Random Walk: Occasionally make a random move, with a certain probability, to escape local minima.
   * Tabu Search: Keep a short memory (tabu list) of recently visited states to avoid cycling back to them.
   * Constraint Weighting: Assign weights to constraints and, at each step, prefer changing variables that are involved in the lowest total weight of violated constraints.